In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
library(dplyr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [2]:
wine_quality <- read_csv(file = "/home/jovyan/dsci-project/DSCI-Project/winequalityN.csv", show_col_types = FALSE)

names(wine_quality) <- make.names(colnames(wine_quality))

white_quality <- filter(wine_quality, type == "white") |>
                select(volatile.acidity, chlorides, total.sulfur.dioxide, quality) |>
                na.omit(volatile.acidity, chlorides, total.sulfur.dioxide, quality)
white_quality$volatile.acidity <- as.numeric(white_quality$volatile.acidity)
white_quality$chlorides<-as.numeric(white_quality$chlorides)
white_quality$total.sulfur.dioxide<-as.numeric(white_quality$total.sulfur.dioxide)
white_quality$quality<-as.numeric(white_quality$quality)

In [3]:
set.seed(135)
wine_split <- initial_split(white_quality, prop = 0.75, strata = quality)
wine_train <- training (wine_split)
wine_test <- testing(wine_split)

In [4]:
#summarize with training data

In [5]:
wine_table <- wine_train|>
    group_by(quality)|>           
    summarize(mean.vol = mean(volatile.acidity), mean.chlor = mean(chlorides), 
              mean.sulf = mean(total.sulfur.dioxide))

wine_table

quality,mean.vol,mean.chlor,mean.sulf
<dbl>,<dbl>,<dbl>,<dbl>
3,0.3291176,0.05535294,169.2059
4,0.4001250,0.05068333,126.7625
5,0.3007202,0.05107064,150.3908
6,0.2615350,0.04503830,137.3249
7,0.2590364,0.03859939,125.1563
8,0.2809470,0.03792424,123.8409
9,0.3150000,0.02800000,104.5000


In [ ]:
#plot with training data